<center style = "font-family: Trebuchet MS, sans-serif;">
    <h1 style = "background-color:#E39DE3; color:white;"> RECOMMENDER SYSTEMS - ASSIGNMENT 01 </h1>
    <hr style = "background-color:#E39DE3;">
</center>
<h3 style = "color:#E39DE3; font-family:Trebuchet MS, sans-serif;"> 
    <b>NAME: </b>Sondos Suhail <br>
    <b>ID: </b>20K-1057
</h3>
<hr style = "background-color:#E39DE3;">
<h3 style = "color:#E39DE3; font-family:Trebuchet MS, sans-serif;">  Imported Libraries </h3>

In [1]:
import pandas as pd
import numpy as np
import math
import nltk

from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

<h3> Task 02 </h3>

<p >Write a Python (or Java) program to implement a Content-based Recommender System that takes a movie name as input and recommends top-3 movies to the user. Use genres as features and the provided ratings for vectorization of all profiles. </p>

<b>UTILITY FUNCTIONS

In [2]:
def split_list(mov_gen):
    gen_lists = []
    for i in mov_gen:
        m = i.split('|')
        gen_lists.append(m)
        
    return gen_lists
    

In [3]:
def remove_year(mov_title):
    x = []
    for i in mov_title:
        y = i[:-6]
        y = y.strip()
        x.append(y)
        
    return x

In [4]:
def stem(text):
    
    ps = PorterStemmer()
    L = []
    for i in text.split():
        L.append(ps.stem(i))
        
    return ' '.join(L)

In [5]:
def recommender(movie_name, movie_df, similarities):
    
    index = np.where(movie_df['title'] == movie_name)[0][0]
    similar_movies = sorted(enumerate(similarities[index]), key=lambda x: x[1], reverse=True)[1:4]
    print(similar_movies)
    movie_list = []
    for i in similar_movies:
        movie_list.append(movies['title'][i[0]])
        
    return movie_list

<b>LOADING ALL DATASETS INTO DATAFRAMES

In [6]:
movies = pd.read_csv(r'D:\FastSemesters\semester6\Recommender_System(E)\A1\ml-latest-small\movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# ratings = pd.read_csv('ratings.csv')
# ratings = ratings.drop('timestamp', axis = 1)
# ratings.head()

<b>CHECKING IF DATASETS HAVE NULL-VALUES

In [8]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [9]:
# ratings.info()

In [10]:
#Spliting single string genres into a list

mov_gen = movies.loc[:, 'genres']
gen_list = split_list(mov_gen)
movies['genres'] = gen_list

movies.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [11]:
#removing years from movie titles
mov_title = movies.loc[:, 'title']
new_list = remove_year(mov_title)
movies['title'] = new_list


movies

,movieId,title,genres
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji,"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men,"[Comedy, Romance]"
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II,[Comedy]
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,"[Action, Animation, Comedy, Fantasy]"
9738,193583,No Game No Life: Zero,"[Animation, Comedy, Fantasy]"
9739,193585,Flint,[Drama]
9740,193587,Bungo Stray Dogs: Dead Apple,"[Action, Animation]"


In [12]:
#converting genres into lowercase
movies['genres'] = movies['genres'].apply(lambda x : str(x).lower())
movies.head()

,movieId,title,genres
0,1,Toy Story,"['adventure', 'animation', 'children', 'comedy..."
1,2,Jumanji,"['adventure', 'children', 'fantasy']"
2,3,Grumpier Old Men,"['comedy', 'romance']"
3,4,Waiting to Exhale,"['comedy', 'drama', 'romance']"
4,5,Father of the Bride Part II,['comedy']


In [13]:
movies['genres'] = movies['genres'].apply(stem)
movies.head()

,movieId,title,genres
0,1,Toy Story,"['adventure', 'animation', 'children', 'comedy..."
1,2,Jumanji,"['adventure', 'children', 'fantasy']"
2,3,Grumpier Old Men,"['comedy', 'romance']"
3,4,Waiting to Exhale,"['comedy', 'drama', 'romance']"
4,5,Father of the Bride Part II,['comedy']


In [14]:
cv = CountVectorizer(max_features = 4000, stop_words = 'english')
movies_data = cv.fit_transform(movies['genres']).toarray()

In [15]:
similarity = cosine_similarity(movies_data)
similarity

array([[1.        , 0.77459667, 0.31622777, ..., 0.        , 0.31622777,
        0.4472136 ],
       [0.77459667, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.31622777, 0.        , 1.        , ..., 0.        , 0.        ,
        0.70710678],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.31622777, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.4472136 , 0.        , 0.70710678, ..., 0.        , 0.        ,
        1.        ]])

In [16]:
recommender('Jumanji', movies, similarity)

[(53, 1.0000000000000002), (109, 1.0000000000000002), (767, 1.0000000000000002)]


['Indian in the Cupboard, The',
 'NeverEnding Story III, The',
 'Escape to Witch Mountain']

In [17]:
recommender('Pocahontas', movies, similarity)

[(618, 0.9999999999999999), (786, 0.8944271909999159), (1390, 0.8451542547285164)]


['Hunchback of Notre Dame, The', 'Dumbo', 'Mulan']

In [18]:
recommender('Beauty and the Beast', movies, similarity)

[(7467, 0.9258200997725515), (780, 0.9128709291752769), (7207, 0.9128709291752769)]


['Tangled', 'Cinderella', 'Princess and the Frog, The']